# Tutorial for AFMpy.REC.HierarchicalDSC

## Imports

In [ ]:
# Standard library imports
import json
import logging

# Third party imports
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import Stack, DL, Plotting, REC

## Configure Logging

In [ ]:
# Load the preconfigured logging settings
with open('logs/HierarchicalDSC_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Config

In [ ]:
Plotting.configure_formatting()

## Load the Stacks

In [ ]:
# Set the filepath for the public key to verify the integrity of the stacks.
PUBLIC_KEY_FILEPATH = '../common/keys/Tutorial_Public.pub'

# Load the cytoplasmic and periplasmic stacks
cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AC-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AP-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

## Prepare the Convolutional Autoencoders

In [ ]:
# Determine the input shape for the CAE. It should be (width,height,channels). In our case (64,64,1)
cyto_input_shape = (*cytoplasmic_stack.stack.shape[1:], 1)
peri_input_shape = (*periplasmic_stack.stack.shape[1:], 1)

# Create the Convolutional Autoencoder models to train with our data.
cytoplasmic_CAE = DL.DefaultCAE(input_shape = cyto_input_shape)
periplasmic_CAE = DL.DefaultCAE(input_shape = peri_input_shape)

## Apply Hierarchical Deep Spectral Clustering

In [ ]:
# Apply Hierarchical DSC to the cytoplasmic stack.
cyto_clusters = REC.hierarchical_DSC(cytoplasmic_stack,
                                     cytoplasmic_CAE,
                                     k_neighbors = 7,
                                     max_iterations = 5,
                                     lafm_target_resolution = (96,96),
                                     lafm_sigma = 2.25,
                                     stability_threshold = 0.85,
                                     distinct_cluster_threshold = 0.7)

# Calculate the mean images for each cluster
for cluster in cyto_clusters:
    cluster.calc_mean_image()
    cluster.calc_LAFM_image(target_resolution = (96, 96), sigma = 2.25)

In [ ]:
fig, ax = plt.subplots(len(cyto_clusters), 2, figsize = (4, len(cyto_clusters) * 2))
for i, cluster in enumerate(cyto_clusters):
    ax[i, 0].imshow(cluster.mean_image, cmap = Plotting.LAFMcmap)
    ax[i, 1].imshow(cluster.LAFM_image, cmap = Plotting.LAFMcmap)

    ax[i,0].set_ylabel(f'Cluster {i}', fontsize = 14)
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])
ax[0, 0].set_title('Mean Image', fontsize = 14)
ax[0, 1].set_title('LAFM Image', fontsize = 14)

In [ ]:
# Apply Hierarchical DSC to the cytoplasmic stack.
peri_clusters = REC.hierarchical_DSC(periplasmic_stack,
                                     periplasmic_CAE,
                                     k_neighbors = 7,
                                     max_iterations = 5,
                                     lafm_target_resolution = (96,96),
                                     lafm_sigma = 2.25,
                                     stability_threshold = 0.85,
                                     distinct_cluster_threshold = 0.7)

# Calculate the mean images for each cluster
for cluster in peri_clusters:
    cluster.calc_mean_image()
    cluster.calc_LAFM_image(target_resolution = (96, 96), sigma = 2.25)

In [ ]:
fig, ax = plt.subplots(len(peri_clusters), 2, figsize = (4, len(peri_clusters) * 2))
for i, cluster in enumerate(peri_clusters):
    ax[i, 0].imshow(cluster.mean_image, cmap = Plotting.LAFMcmap)
    ax[i, 1].imshow(cluster.LAFM_image, cmap = Plotting.LAFMcmap)

    ax[i,0].set_ylabel(f'Cluster {i}', fontsize = 14)
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])
ax[0, 0].set_title('Mean Image', fontsize = 14)
ax[0, 1].set_title('LAFM Image', fontsize = 14)